In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

# Load the input dataset
df = pd.read_csv('summary_table_supervised_PT_PP_PT_CTRL_and_PT_PP_PT_CTRL_long_7_9_12_230524.txt', sep='\t')
df.drop(columns=['Chr', 'Start', 'End', 'GeneLength'], inplace=True)

# Compute log2(Fold Change) and -log10(p-value) for each time point
for day in [7, 9, 12]:
    df[f'-log10(p){day}'] = -np.log10(df[f'ttest_unpaired_p_PT_PP_PT_CTRL_long_{day}'])
    df[f'log2(FC){day}'] = np.log2(df[f'folds_median_PT_PP_PT_CTRL_long_{day}'])

# Define a function to classify gene regulation status
def assign_regulation(logfc, logp):
    if logp > 1.3:
        if logfc >= 1:
            return 'Increased'
        elif logfc <= -1:
            return 'Decreased'
    return 'Neutral'

# Create the Dash application
app = Dash(__name__)
app.layout = html.Div([
    html.H2("Volcano plot – Days of PP Treatment", style={'textAlign': 'center'}),
    dcc.Dropdown(
        id='day-selector',
        options=[{'label': f'{day} days', 'value': day} for day in [7, 9, 12]],
        value=9,
        clearable=False
    ),
    dcc.Graph(id='volcano-plot')
])

# Define callback to update volcano plot based on selected time point
@app.callback(
    Output('volcano-plot', 'figure'),
    Input('day-selector', 'value')
)
def update_volcano_plot(day):
    temp_df = df.copy()
    temp_df['log2FC'] = temp_df[f'log2(FC){day}']
    temp_df['-log10p'] = temp_df[f'-log10(p){day}']
    temp_df['Regulation'] = temp_df.apply(lambda row: assign_regulation(row['log2FC'], row['-log10p']), axis=1)

    fig = px.scatter(
        temp_df,
        x='log2FC',
        y='-log10p',
        color='Regulation',
        color_discrete_map={
            'Increased': '#AF4647',
            'Decreased': '#517FBC',
            'Neutral': 'lightgray'
        },
        hover_name='GeneName',
        title=f'Volcano Plot – {day} Days of PP Treatment',
        height=700,
        width=800
    )

    # Add threshold lines for significance
    fig.add_hline(y=1.3, line_dash='dash', line_color='gray')
    fig.add_vline(x=1, line_dash='dash', line_color='gray')
    fig.add_vline(x=-1, line_dash='dash', line_color='gray')

    fig.update_layout(
        xaxis_title='log₂(Fold Change)',
        yaxis_title='-log(p)',
        font=dict(size=14),
        legend=dict(title='Regulation'),
        plot_bgcolor='white'
    )

    return fig
    
# Run the app
if __name__ == '__main__':
    app.run(debug=True, port=8051)